In [46]:
import json
import urllib2
import time
import networkx as nx
import matplotlib.pyplot as plt

G=nx.Graph()
ids = set()

#ids.add(28585277)
friends_request = 'https://api.vk.com/method/friends.get?user_id=28585277'
data = json.load(urllib2.urlopen(friends_request))
for x in data['response']:
    ids.add(x)
for x in data['response']:
    data1 = json.load(urllib2.urlopen('https://api.vk.com/method/friends.get?user_id='+str(x)))
    time.sleep(1)
    try:
        common = set(data1['response']).intersection(ids)
        for y in common:
            G.add_edge(x,y)
            with open('friends.txt', 'a') as f:
                f.write(str(x)+' '+str(y)+'\n')
    except:
        pass

In [47]:
token = '6388ee8a65fc9ef610df6e010aab8fa1fbd2f372e94b676b591626cbecaf692104415fafa2feb29313bca'
uni = {}
dcity = {}
has_university = []
has_city = []
for node in G.nodes():
    request = 'https://api.vk.com/method/users.get?user_id='+str(node)+'&fields=education,city&access_token='+token
    city = 0
    code = 0
    try:
        data = json.load(urllib2.urlopen(request))
        code = data['response'][0][u'university']
        city = data['response'][0][u'city']
    except:
        code = 0
        city = 0
    uni[node] = code
    dcity[node] = city
    if code:
        has_university.append(node)
    if city:
        has_city.append(node)
    time.sleep(0.4)
nx.set_node_attributes(G, 'university', uni)
nx.set_node_attributes(G, 'city', dcity)

In [48]:
nx.set_node_attributes(G, 'degree_centrality', nx.degree_centrality(G))

In [49]:
nx.attribute_assortativity_coefficient(G,'university',has_university)

0.20526249285077763

In [50]:
nx.attribute_assortativity_coefficient(G,'city',has_city)

0.11829128584913325

Similarity mesures:

In [89]:
import numpy as np
jaccard = np.zeros((G.number_of_nodes(), G.number_of_nodes()))
mapping = {}
count = 0
for x in G.nodes():
    mapping[x]=count
    count+=1
for x in G.nodes():
    for y in G.nodes():
        a = len(set(G.neighbors(x)).intersection(set(G.neighbors(y))))
        b = len(set(G.neighbors(x)).union(set(G.neighbors(y))))
        jaccard[mapping[x],mapping[y]] = 1.0*a/b
        if (x,y) in G.edges():
            G[x][y]['jaccard'] = 1.0*a/b

cosine = np.zeros((G.number_of_nodes(), G.number_of_nodes()))
for x in G.nodes():
    for y in G.nodes():
        a = len(set(G.neighbors(x)).intersection(set(G.neighbors(y))))
        b = G.degree(x)
        c = G.degree(y)
        cosine[mapping[x],mapping[y]] = 1.0*a/pow(b*c,0.5)
        if (x,y) in G.edges():
            G[x][y]['cosine'] = 1.0*a/pow(b*c,0.5)

In [94]:
plt.matshow(cosine)
plt.colorbar()
plt.show()

Multilevel recursive partitioning algorithm with modularity thresholding (from homework):

In [56]:
import scipy.io
import scipy.sparse as ss

clusters = []

def modularity1(H, c1, c2):
    edges = H.edges()
    nodes = H.nodes()
    m = H.number_of_edges()
    Q = 0
    for a in nodes:
        for b in nodes:
            x=0
            if (a,b) in edges:
                x=1
            d = 0
            if ((a in c1) and (b in c1)) or ((a in c2) and (b in c2)):
                d = 1
            Q += (x-H.degree(a)*H.degree(b)/(2.0*m))*d
    return Q/(2*m)

def spectral(c, threshold):
    H = G.subgraph(c)
    mapping = {}
    count = 0
    for x in H.nodes():
        mapping[count]=x
        count+=1
    
    A = nx.adjacency_matrix(H).asfptype()
    L = nx.laplacian_matrix(H).asfptype()
    D = L + A
    w, v = ss.linalg.eigsh(L, k=2,  M = D, which = 'SM')
    c1 = []
    c2 = []
    for i in range(len(v[:, 1])):
        if v[i, 1]>0:
            c1.append(mapping[i])
        else:
            c2.append(mapping[i])
    if modularity1(H,c1,c2)>threshold:
        for i in range(len(clusters)):
            if set(c1) in set(clusters[i]):
                del clusters[i]
                break
        clusters.append(c1)
        for i in range(len(clusters)):
            if set(c2) in set(clusters[i]):
                del clusters[i]
                break
        clusters.append(c2)
        return 0
    spectral(c1,threshold)
    spectral(c2,threshold)

In [57]:
spectral(G.nodes(),-0.2)
spec_clust = {}
for i in range(len(clusters)):
    for node in clusters[i]:
        spec_clust[node] = i
nx.set_node_attributes(G, 'spectral', spec_clust)
import community
community.modularity(spec_clust, G)

0.39194564378914115

Louvain algorithm (community module):

In [58]:
partition = community.best_partition(G)
nx.set_node_attributes(G, 'louvain', partition)
community.modularity(partition, G)

0.40617631037294033

Core and cliques:

In [59]:
H = nx.k_core(G)
main_core = {}
for node in G.nodes():
    main_core[node] = 0
    if node in H.nodes():
        main_core[node] = 1
nx.set_node_attributes(G, 'core', main_core)

In [95]:
max_clique={}
all_cliques = list(nx.find_cliques(G))
print len(all_cliques)
for y in all_cliques:
        if len(y)==14:
            print '1'
max_union = 0
a = []
b = []
for x in all_cliques:
    for y in all_cliques:
        if len(x)==14 and len(y)==14:
            if len(set(x).union(set(y)))>max_union:
                max_union = len(set(x).union(set(y)))
                a = x
                b = y
for node in G.nodes():
    max_clique[node] = 0
    if node in a:
        max_clique[node] = 1
    if node in b:
        max_clique[node] = 2
    if (node in b) and (node in a):
        max_clique[node] = 3 
nx.set_node_attributes(G, 'clique', max_clique)

142
1
1
1
1
1
1
1
1


In [92]:
nx.write_gexf(G, "mynetwork.gexf")